### Steps for execution:
- 1) Import the Rubric which has all the matching EOM fee amounts 
- 2) Import the EOM View dataframe, this is the base of our output
- 3) Needed column from EOM: Processor, Card Type, Merchant Group, Attempted Captured Charges, Processed, Chargebacks, Alerts.
- 4) Columns to calc: Disc Due, Auth Due, CB Due, Visa Alert Due
    - Disc Due --> **Processed** x **Discount Fee**
    - Auth Due --> **Attempted Captured Charges** x **Attemt Fees**
    - CB Due --> **Chargebacks** x **35**
    - Visa Alert Due --> **Alerts** x **Visa Alert**
    - EOM --> **SUM OF ALL**


### Step 1) Import the Rubric Dataframe which we will convert to a dictionary

In [2]:
import os
import pandas as pd 
import numpy as np

# Get the current directory (where the notebook is saved)
current_directory = os.getcwd()

files = os.listdir(current_directory)

# Get the EOM df
for RUBRIC_df in files:
    if "EOM" in RUBRIC_df and "Rubric" in RUBRIC_df:
        print(RUBRIC_df)
        break

# Import the EOM_df 
import pandas as pd
import os

def import_file(file_name):
    """
    Import a specific file regardless of whether it's CSV or Excel
    
    Parameters:
    file_name (str): Name of the file to import
    
    Returns:
    pandas.DataFrame: The imported data
    """
    # Get the full path
    file_path = os.path.join(os.getcwd(), file_name)
    
    # Check if file exists
    if not os.path.exists(file_path):
        raise FileNotFoundError(f"File {file_name} not found in the current directory")
    
    # Get the file extension (lowercase)
    _, file_extension = os.path.splitext(file_name)
    file_extension = file_extension.lower()
    
    # Import based on file extension
    if file_extension == '.csv':
        return pd.read_csv(file_path)
    elif file_extension in ['.xlsx', '.xls']:
        return pd.read_excel(file_path)
    else:
        raise ValueError(f"Unsupported file format: {file_extension}")

# Example usage:
try:
    RUBRIC_df = import_file(RUBRIC_df)
    print(f"Successfully imported {RUBRIC_df}")
except Exception as e:
    print(f"Error importing file: {e}")
RUBRIC_df.columns = RUBRIC_df.iloc[1]
RUBRIC_df = RUBRIC_df[2:]
RUBRIC_df

# restaret the index 
RUBRIC_df.reset_index(drop=True, inplace=True)

# make sure that the Processor column is a string 
RUBRIC_df['Processor'] = RUBRIC_df['Processor'].astype(str)

RUBRIC_df

GF_EOM-Rubric.xlsx
Successfully imported           EOM Rubric     Unnamed: 1    Unnamed: 2 Unnamed: 3      Unnamed: 4
0                NaN            NaN           NaN        NaN             NaN
1          Processor  Discount Fees  Attempt Fees     CB Fee  Visa Alert Due
2            PAYSAFE              0           0.6         35              20
3             PAYARC              0             0          0               0
4               APPS              0             0          0               0
5            NETEVIA              0           0.5          0               0
6           PRIORITY            NaN           NaN        NaN             NaN
7            QUANTUM           0.07           0.6         35              15
8           SignaPay              0          0.65         35              20
9           Maverick              0           0.5         35              20
10      PAYSAFE_BBVA              0           0.6         35              20
11       PAYSAFE_PNC              0

1,Processor,Discount Fees,Attempt Fees,CB Fee,Visa Alert Due
0,PAYSAFE,0,0.6,35,20
1,PAYARC,0,0,0,0
2,APPS,0,0,0,0
3,NETEVIA,0,0.5,0,0
4,PRIORITY,NaN,NaN,NaN,NaN
5,QUANTUM,0.07,0.6,35,15
6,SignaPay,0,0.65,35,20
7,Maverick,0,0.5,35,20
8,PAYSAFE_BBVA,0,0.6,35,20
9,PAYSAFE_PNC,0,0.6,35,20


In [3]:
# make sure that the remainig column are in float 
RUBRIC_df.iloc[:, 1:] = RUBRIC_df.iloc[:, 1:].astype(float)
what = RUBRIC_df.loc[5, "Processor"]

# We are goiung to convert the RUBRIC_df into a dictionary
RUBRIC_dict = RUBRIC_df.set_index("Processor").T.to_dict()

# Set all values into float 
RUBRIC_dict = {
    key.lower(): {inner_key: float(inner_value) for inner_key, inner_value in value.items()}
    for key, value in RUBRIC_dict.items()
}

RUBRIC_dict = {key.lower(): value for key, value in RUBRIC_dict.items()}
RUBRIC_dict["merchant industries"] = {
    "Discount Fees": 0.0,
    "Attempt Fees": 0.0,
    "CB Fee": 35.0,
    "Visa Alert Due": 0.0
}

RUBRIC_dict

C:\Users\mmsou\AppData\Local\Temp\ipykernel_34328\4245926676.py:6: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  RUBRIC_dict = RUBRIC_df.set_index("Processor").T.to_dict()


{'paysafe': {'Discount Fees': 0.0,
  'Attempt Fees': 0.6,
  'CB Fee': 35.0,
  'Visa Alert Due': 20.0},
 'payarc': {'Discount Fees': 0.0,
  'Attempt Fees': 0.0,
  'CB Fee': 0.0,
  'Visa Alert Due': 0.0},
 'apps': {'Discount Fees': 0.0,
  'Attempt Fees': 0.0,
  'CB Fee': 0.0,
  'Visa Alert Due': 0.0},
 'netevia': {'Discount Fees': 0.0,
  'Attempt Fees': 0.5,
  'CB Fee': 0.0,
  'Visa Alert Due': 0.0},
 'priority': {'Discount Fees': nan,
  'Attempt Fees': nan,
  'CB Fee': nan,
  'Visa Alert Due': nan},
 'quantum': {'Discount Fees': 0.07,
  'Attempt Fees': 0.6,
  'CB Fee': 35.0,
  'Visa Alert Due': 15.0},
 'signapay': {'Discount Fees': 0.0,
  'Attempt Fees': 0.65,
  'CB Fee': 35.0,
  'Visa Alert Due': 20.0},
 'maverick': {'Discount Fees': 0.0,
  'Attempt Fees': 0.5,
  'CB Fee': 35.0,
  'Visa Alert Due': 20.0},
 'paysafe_bbva': {'Discount Fees': 0.0,
  'Attempt Fees': 0.6,
  'CB Fee': 35.0,
  'Visa Alert Due': 20.0},
 'paysafe_pnc': {'Discount Fees': 0.0,
  'Attempt Fees': 0.6,
  'CB Fee': 3

### Step 2) Import the EOM dataframe

In [4]:
# Get the EOM df
for EOM_df in files:
    if "EOM" in EOM_df and "View" in EOM_df:
        print(EOM_df)
        break

In [5]:
current_directory = os.getcwd()
current_directory

'c:\\Users\\mmsou\\Documents\\mokum.ai\\Goldfinger'

In [6]:
import pandas as pd
import os

# Get the current directory (where the notebook is saved)
current_directory = os.getcwd()

# List all files in the current directory
files = os.listdir(current_directory)

# Import the EOM_df
def import_file(file_name):
    if file_name.endswith(".csv"):
        return pd.read_csv(file_name)
    elif file_name.endswith((".xls", ".xlsx")):
        return pd.read_excel(file_name)
    else:
        raise ValueError("Unsupported file format")

# Example usage: Look for a file named "EOM-View_EXPORT.csv" or similar
for file in files:
    if "EOM" in file and file.endswith((".csv", ".xls", ".xlsx")) and "Fcast" in file:
        EOM_df = import_file(file)
        break
else:
    raise FileNotFoundError("No EOM file found in the current directory")

# Convert "Processor", "Card Type", and "Merchant Group" into string columns
EOM_df["Processor"] = EOM_df["Processor"].astype(str)
EOM_df["Card Type"] = EOM_df["Card Type"].astype(str)
EOM_df["Merchant Group"] = EOM_df["Merchant Group"].astype(str)


- Disc Due --> **Processed** x **Discount Fee**
- Auth Due --> **Attempted Captured Charges** x **Attempt Fees**
- CB Due --> **Chargebacks** x **35**
- Visa Alert Due --> **Alerts** x **Visa Alert**
- EOM --> **SUM OF ALL**

As we through the df we append the index and values for each dict representing each new colummn, in the end we will map the original df with these to fill them out

In [7]:
EOM_df = EOM_df[EOM_df['Merchant Group'] != "Sale Shield"]
EOM_df = EOM_df[EOM_df['Merchant Group'] != "SaleShield"]
EOM_df = EOM_df[EOM_df["Processor"] != "EMS"]

In [8]:
EOM_df.columns

# Columns to keep Processor, Card Type, Merchant Group, Attempted Captured Charges, Processed, Chargebacks, Alerts, Disc Due, Auth Due, CB Due, Visa Alert Due
EOM_df = EOM_df[["Processor", "Card Type", "Merchant Group", "Attempted Captured Charges", "Processed", "Chargebacks", "Alerts"]]
EOM_df["Disc Due"] = np.nan
EOM_df["Auth Due"] = np.nan
EOM_df["CB Due"] = np.nan
EOM_df["Visa Alert Due"] = np.nan
EOM_df["Total EOM"] = np.nan

# Drop all rows that have FlexFactor and Stripe in the processor column
EOM_df = EOM_df[~EOM_df["Processor"].str.contains("FlexFactor|Stripe", na=False)]

# Remove trailing spaces from EOM_df column names
def clean_column_names(df):
    df.columns = (
        df.columns.str.strip()         # Remove leading/trailing spaces
        .str.lower()                   # Convert to lowercase
        .str.replace(r'\W+', '_', regex=True)  # Replace non-word characters with '_'
        .str.replace(r'_+', '_', regex=True)   # Remove multiple consecutive '_'
        .str.rstrip('_')                # Remove trailing '_'
    )
    return df

# Clean Porcessed column such that we remove special characters and convert to float
EOM_df["Processed"] = EOM_df["Processed"].astype(str).str.replace(r'[^0-9.]', '', regex=True).astype(float)

# Replace empty strings and non-numeric values with NaN
EOM_df["Attempted Captured Charges"] = (
    EOM_df["Attempted Captured Charges"]
    .astype(str)  # Ensure the column is treated as strings
    .str.replace(r'[^0-9.]', '', regex=True)  # Remove non-numeric characters
    .replace('', 0)  # Replace empty strings with NaN
)

# Convert the column to float, coercing any remaining invalid values to NaN
EOM_df["Attempted Captured Charges"] = pd.to_numeric(EOM_df["Attempted Captured Charges"], errors='coerce')

# Convert charbacks column into integer 
# Clean and convert the column (replace "Chargebacks" with the desired column name)
EOM_df["Chargebacks"] = (
    EOM_df["Chargebacks"]
    .astype(str)  # Ensure the column is treated as strings
    .str.replace(r'[^0-9.]', '', regex=True)  # Remove non-numeric characters
    .replace('', 0)  # Replace empty strings with NaN
    .astype(float)  # Convert to float
)
EOM_df["Chargebacks"] = EOM_df["Chargebacks"].astype(str).str.replace(r'[^0-9.]', '', regex=True).astype(float)

EOM_df = clean_column_names(EOM_df)


In [15]:
# All teh rpocessors from the rubric 
processors = RUBRIC_df["Processor"].unique()
processors

processors_in_EOM = EOM_df["processor"].unique()

# now we need to check if all the processors in the rubric are in the EOM df
print("Processors in Rubric:")
print(processors)
print("\nProcessors in EOM:")
print(processors_in_EOM)

# Check for missing processors
missing_processors = []
for processor in processors:
    if processor.lower() not in [p.lower() for p in processors_in_EOM]:
        missing_processors.append(processor)

if missing_processors:
    print(f"\n⚠️ Warning: The following processors from the rubric are not found in EOM data:")
    for proc in missing_processors:
        print(f"  - {proc}")
else:
    print("\n✅ All processors from the rubric are present in the EOM data")

# Check for processors in EOM that are not in the rubric
extra_processors = []
for processor in processors_in_EOM:
    if processor.lower() not in [p.lower() for p in processors]:
        extra_processors.append(processor)

if extra_processors:
    print(f"\n⚠️ Warning: The following processors in EOM data are not in the rubric:")
    for proc in extra_processors:
        print(f"  - {proc}")
else:
    print("\n✅ All processors in EOM data are covered by the rubric")

# Now let's calculate the fees for each row
print("\n🔄 Calculating fees for each row...")

# Initialize counters for tracking
processed_rows = 0
error_rows = 0
errors = []

for index, row in EOM_df.iterrows():
    try:
        processor = row['processor'].lower()
        
        # Get the fee structure for this processor
        if processor in RUBRIC_dict:
            fees = RUBRIC_dict[processor]
        else:
            # Try to find a partial match
            matching_processor = None
            for rubric_proc in RUBRIC_dict.keys():
                if processor in rubric_proc or rubric_proc in processor:
                    matching_processor = rubric_proc
                    break
            
            if matching_processor:
                fees = RUBRIC_dict[matching_processor]
                print(f"  📝 Using fees from '{matching_processor}' for processor '{processor}'")
            else:
                # Use default fees for unknown processors
                fees = {
                    "Discount Fees": 0.0,
                    "Attempt Fees": 0.0,
                    "CB Fee": 35.0,
                    "Visa Alert Due": 0.0
                }
                print(f"  ⚠️ No fee structure found for processor '{processor}', using defaults")
        
        # Calculate fees
        processed_amount = row['processed'] if pd.notna(row['processed']) else 0
        attempted_charges = row['attempted_captured_charges'] if pd.notna(row['attempted_captured_charges']) else 0
        chargebacks = row['chargebacks'] if pd.notna(row['chargebacks']) else 0
        alerts = row['alerts'] if pd.notna(row['alerts']) else 0
        
        # Calculate each fee type
        disc_due = processed_amount * fees["Discount Fees"]
        auth_due = attempted_charges * fees["Attempt Fees"]
        cb_due = chargebacks * fees["CB Fee"]
        visa_alert_due = alerts * fees["Visa Alert Due"]
        total_eom = disc_due + auth_due + cb_due + visa_alert_due
        
        # Update the row
        EOM_df.at[index, 'disc_due'] = disc_due
        EOM_df.at[index, 'auth_due'] = auth_due
        EOM_df.at[index, 'cb_due'] = cb_due
        EOM_df.at[index, 'visa_alert_due'] = visa_alert_due
        EOM_df.at[index, 'total_eom'] = total_eom
        
        processed_rows += 1
        
    except Exception as e:
        error_rows += 1
        errors.append(f"Row {index}: {str(e)}")
        print(f"  ❌ Error processing row {index}: {str(e)}")

print(f"\n✅ Processing complete:")
print(f"  - Successfully processed: {processed_rows} rows")
print(f"  - Errors: {error_rows} rows")

if errors:
    print("\n❌ Errors encountered:")
    for error in errors[:5]:  # Show first 5 errors
        print(f"  {error}")
    if len(errors) > 5:
        print(f"  ... and {len(errors) - 5} more errors")

# Display summary statistics
print("\n📊 Summary Statistics:")
print(f"Total EOM fees: ${EOM_df['total_eom'].sum():,.2f}")
print(f"Average EOM per row: ${EOM_df['total_eom'].mean():,.2f}")
print(f"Max EOM per row: ${EOM_df['total_eom'].max():,.2f}")
print(f"Min EOM per row: ${EOM_df['total_eom'].min():,.2f}")

# Show breakdown by fee type
print("\n💰 Fee Breakdown:")
print(f"Discount fees: ${EOM_df['disc_due'].sum():,.2f}")
print(f"Authorization fees: ${EOM_df['auth_due'].sum():,.2f}")
print(f"Chargeback fees: ${EOM_df['cb_due'].sum():,.2f}")
print(f"Visa Alert fees: ${EOM_df['visa_alert_due'].sum():,.2f}")

# Display the final dataframe
print("\n📋 Final EOM DataFrame:")
EOM_df.head(10)

Processors in Rubric:
['PAYSAFE' 'PAYARC' 'APPS' 'NETEVIA' 'PRIORITY' 'QUANTUM' 'SignaPay'
 'Maverick' 'PAYSAFE_BBVA' 'PAYSAFE_PNC' 'APPS_SYNOVOUS' 'APPS_SYNOVUS'
 'NETEVIA_ESQUIRE' 'PAYARC_EVOLVE' 'PRIORITY_SYNOVUS' 'QUANTUM_CBSL'
 'QUANTUM_FRESNO' 'SIGNAPAY' 'PayArc']

Processors in EOM:
['APPS' 'Luqra' 'NETEVIA' 'PAYARC' 'PAYSAFE' 'PRIORITY' 'PayArc' 'QUANTUM'
 'SIGNAPAY']

⚠️ Warning: The following processors from the rubric are not found in EOM data:
  - Maverick
  - PAYSAFE_BBVA
  - PAYSAFE_PNC
  - APPS_SYNOVOUS
  - APPS_SYNOVUS
  - NETEVIA_ESQUIRE
  - PAYARC_EVOLVE
  - PRIORITY_SYNOVUS
  - QUANTUM_CBSL
  - QUANTUM_FRESNO

⚠️ Warning: The following processors in EOM data are not in the rubric:
  - Luqra

🔄 Calculating fees for each row...
  ⚠️ No fee structure found for processor 'luqra', using defaults
  ⚠️ No fee structure found for processor 'luqra', using defaults

✅ Processing complete:
  - Successfully processed: 66 rows
  - Errors: 0 rows

📊 Summary Statistics:
Total EOM f

,processor,card_type,merchant_group,attempted_captured_charges,processed,chargebacks,alerts,disc_due,auth_due,cb_due,visa_alert_due,total_eom
0,APPS,Mastercard,SpecifiConLLC,723,10684.75,6.0,69,0.0,0.0,0.0,0.0,0.0
1,APPS,Visa,SpecifiConLLC,1712,9322.38,0.0,49,0.0,0.0,0.0,0.0,0.0
2,Luqra,Mastercard,nan,598,6121.55,0.0,0,0.0,0.0,0.0,0.0,0.0
3,Luqra,Visa,nan,1165,3547.20,0.0,0,0.0,0.0,0.0,0.0,0.0
4,NETEVIA,Mastercard,BrightAdvantageLLC,979,15005.69,17.0,74,0.0,489.5,0.0,0.0,489.5
5,NETEVIA,Mastercard,DiamondSphereLLC,1100,19793.71,8.0,92,0.0,550.0,0.0,0.0,550.0
6,NETEVIA,Mastercard,PrimeSmartSolutionsLLC,671,15238.63,4.0,54,0.0,335.5,0.0,0.0,335.5
7,NETEVIA,Mastercard,SpecifiConLLC,912,14131.09,5.0,69,0.0,456.0,0.0,0.0,456.0
8,NETEVIA,Mastercard,UniquePlusLLC,646,12473.69,6.0,57,0.0,323.0,0.0,0.0,323.0
9,NETEVIA,Visa,BrightAdvantageLLC,2099,8747.87,0.0,20,0.0,1049.5,0.0,0.0,1049.5


In [14]:
# If nan in the processor column, then drop the row
EOM_df = EOM_df.dropna(subset=["processor"])
# Reset the index
EOM_df.reset_index(drop=True, inplace=True)
EOM_df 

,processor,card_type,merchant_group,attempted_captured_charges,processed,chargebacks,alerts,disc_due,auth_due,cb_due,visa_alert_due,total_eom
0,APPS,Mastercard,SpecifiConLLC,723,10684.75,6.0,69,NaN,NaN,NaN,NaN,NaN
1,APPS,Visa,SpecifiConLLC,1712,9322.38,0.0,49,NaN,NaN,NaN,NaN,NaN
2,Luqra,Mastercard,nan,598,6121.55,0.0,0,NaN,NaN,NaN,NaN,NaN
3,Luqra,Visa,nan,1165,3547.20,0.0,0,NaN,NaN,NaN,NaN,NaN
4,NETEVIA,Mastercard,BrightAdvantageLLC,979,15005.69,17.0,74,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
61,SIGNAPAY,Mastercard,UniquePlusLLC,353,6424.23,5.0,43,NaN,NaN,NaN,NaN,NaN
62,SIGNAPAY,Visa,BrightAdvantageLLC,1043,7169.02,3.0,25,NaN,NaN,NaN,NaN,NaN
63,SIGNAPAY,Visa,DiamondSphereLLC,1287,7822.26,1.0,21,NaN,NaN,NaN,NaN,NaN
64,SIGNAPAY,Visa,SpecifiConLLC,101,637.97,6.0,1,NaN,NaN,NaN,NaN,NaN
